# Algorithms of the Mind

**Instructions:** Answer all questions below. Be sure to show all intermediate steps and equations that you used to arrive at each answer. Please type your answers (including your equations). For coding questions, your code and its execution will do.

**How to submit?:** Execute all blocks of your Jupyter notebook, save it, and submit your assignment using Canvas.

<div class="alert alert-info">
    <strong>Note</strong>

Your answers in each question can be a combination of markdown and Julia code.
</div>

## Preliminaries

In [ ]:
NAME = ""
NETID = ""

Next, please take the honor pledge by reordering the following phrases so that it makes sense to you, and then typing the resulting full sentence.

- and that this work is my own.
- or received
- I have not given
- I affirm that
- on this assignment,
- any unauthorized help 

In [ ]:
HONOR_PLEDGE = ""

---

# Problem Set 4

This problem set will be unusual in that it'll be half a lab section and half a problem set.

The first half (or so) of the problem set will feel lab-like because it introduces creating and solving MDPs using the `POMDPs.jl` package (an incredibly done package!).
Critically, this lab-like structure will be necessary for the remainder of the problem set, and you will receive many of your points merely for going through the material.

<!-- It'll feel like a lab section in that there is a good portion of the problem set that introduces how you can create and solve a markov decision problem using the POMDPs.jl package in Julia (a very well done package!). Critically, this will be worth the effort: Rest of the problem set depends on it, and you will get a good chunk of your points for merely going through that material. -->

The second half (or so) of the problem set will feel more pset-like as you will be tasked with creating generative models and making inference using it.
Like previous problem sets, you will be graded on these models and your inferences as well.

Enjoy the problem set &ndash; we think it'll be fun!

In [ ]:
import Pkg
Pkg.activate("psyc261")
Pkg.add([
    "Distributions",
    "POMDPs",
    "POMDPTools",
    "POMDPPolicies",
    "POMDPSimulators",
    "DiscreteValueIteration",
    "Compose",
    "StaticArrays",
    "ColorSchemes",
])
# load necessary packages for this problem set
# Note that running this for the first time might take a good 15 mins; plan ahead
using Random
using Gen
using Plots
using DelimitedFiles
using POMDPs, POMDPTools, POMDPPolicies, POMDPSimulators
using Compose
using StaticArrays
using ColorSchemes

## Question 1: Inferring rewards in an MDP

We will work on a maze-navigation problem in a grid-world domain.
As this is a grid-world MDP, the agent **fully observes** its location (a coordinate-pair).
The agent may stochastically move in one of the cardinal directions.
Lastly, the agent may receive a positive or negative reward for entering particular states.

In more technical terms, the MDP's $\langle \mathcal{S, A, T, R} \rangle$ tuple is:
- $\mathcal{S}$: each grid-world cell,
- $\mathcal{A}$: are cardinal directions (`N`, `E`, `S`, `W`).
- $\mathcal{T}$: transitions are stochastic ($0.9$ probability of moving as intended, $0.1$ probability of moving randomly),
- $\mathcal{R}$: cells may either contain $0$ reward, a positive reward, or a negative reward.

As an observer, we watch the agent's actions, we know the locations of the rewards, and know the values of the negative rewards; **however**, we _do not know the values of the **positive rewards**_.

<div class="alert alert-success" markdown="1">

Your goal is to infer the positive reward values from the agent's actions.
</div>

We can formalize this maze-navigation problem using the MDP framework to setup a generative model of agents, where our latent random variables will be the positive reward values.

### 1A [5 pts] Defining the MDP for our grid world

<div class="alert alert-warning" markdown="1">
    <strong>Note:</strong>

You will not be asked to write code for this question. Your job is to process the following material (in conjuction with the tutorial linked below, as needed) and respond to a brief comprehension question at the end.
</div>

We will use [`POMDPs.jl`](https://github.com/JuliaPOMDP/POMDPs.jl) to setup our MDP and solve it (using the Value Iteration algorithm). You can find documentation for `POMDPs.jl` [here](https://juliapomdp.github.io/POMDPs.jl/stable/).

We will closely follow [one of the tutorials provided by `POMDPs.jl`][gw-tutorial] (which you are free to consult as you like).

<div class="alert alert-warning" markdown="1">
    <strong>Note:</strong>

The tutorial above references some of `POMDPs.jl`'s older packages/APIs, we have taken steps to ensure that our notebook uses the correct, and latest, APIs/packages &ndash; so the tutorial should be referred to primarily for understanding, rather than checking your code against.
</div>

[gw-tutorial]: https://github.com/JuliaPOMDP/POMDPExamples.jl/blob/master/notebooks/GridWorld.ipynb

First, we will create a `struct` to represent the world state. Notice that this struct contains information only about the aspects of the grid world that changes at each time step: the location of the agent and whether it is at a terminal state. 

In [ ]:
const Point = SVector{2,Int}

struct GridWorldState
    pos::Point  # an (x, y) coordinate pair
    done::Bool  # are we in a terminal state?
end

# a helper function as a state constructor
GridWorldState(x::Int, y::Int) = GridWorldState(Point(x, y), false)
;

Next, we will create a `struct` to represent our actions.
This is primarily for syntactic sugar.

In [ ]:
struct GridWorldAction
    pos::Point  # an (x, y) coordinate pair, exactly like in `GridWorldState`
end

# a helper function as an action constructor
GridWorldAction(x::Int, y::Int) = GridWorldAction(Point(x, y))

const N = GridWorldAction(0, 1)
const E = GridWorldAction(1, 0)
const S = GridWorldAction(0, -1)
const W = GridWorldAction(-1, 0)
;

Now we define our MDP problem using a `mutable struct`.
To do this, our `mutable struct` must inherit from the `MDP` type and we need to ensure that the "state" and "action" types are present as well.

Our `GridWorld` contains the dimensions of the grid-world (`size_x`, `size_y`) (which determines the size of the state space), a list of states with some reward (`reward_states`), and the reward values for each of those states (`reward_values`).

<div class="alert alert-warning" markdown="1">
    <strong>Note</strong>

States with a positive reward are designed to be terminal nodes. After visiting one of these states, the agent cannot further accumulate reward.
</div>

Additionally, the `tprob` parameter controls the stochasticity of the environment: specifically, `tprob` is the probability the agent's intended action succeeds (e.g., a move `N` indeed moves `:N`), while `1 - tprob` is the probability the agent moves randomly.
In this problem set, we set `tprob = 0.9`.
This stochasticity will be an important variable to account for when writing our likelihood function in [Q1B](#q1b).

Lastly, our MDP (as with all MDPs) must include a `discount_factor` ($\mathcal{\gamma}$).
Discount factors (or rates) can be used to incentivize agents to maximize reward over shorter paths.
For our purposes, we will use `discount_factor = 0.9`, this means the future reward estimates will be diminished by 10% for each step into the future they are. (This is much like placing a negative reward on each cell for simply taking a step, as discussed in the lecture.)

In [ ]:
# the Grid World MDP
# Note that our MDP is parameterized by the state and action
mutable struct GridWorld <: MDP{GridWorldState,GridWorldAction}
    size_x::Int # x size of the grid
    size_y::Int # y size of the grid
    reward_states::Vector{GridWorldState} # the states in which agent recieves reward
    reward_values::Vector{Real} # reward values for those states
    tprob::Real # probability of transitioning to the desired state
    discount_factor::Real # disocunt factor
end

Now we will define a helper function to construct a GridWorld MDP with some default values, and then call it.

In [ ]:
#we use key worded arguments so we can change any of the values we pass in 
function GridWorld(;
    sx::Int=10, # size_x
    sy::Int=10, # size_y
    rs::Vector{GridWorldState}=[  # reward states
        GridWorldState(Point(4, 3), true),
        GridWorldState(Point(4, 6), true),
        GridWorldState(Point(8, 2), true),
        GridWorldState(Point(4, 9), true),
    ],
    rv::Vector{<:Real}=[-10.0, -5.0, 5.0, 5.0], # reward values
    tp::Real=0.9, # tprob
    discount_factor::Real=0.9
)
    return GridWorld(sx, sy, rs, rv, tp, discount_factor)
end

# we can now instantiate a GridWorld `mdp` like this:
mdp = GridWorld(; rv=[-10, -5.0, 2, 10]);
# you can view the reward states by uncommenting the following line:
#mdp.reward_states # mdp contains all the defualt values from the constructor

Now with the `mdp::GridWorld` at hand, we can fully specify the tuple $\langle \mathcal{S, A, T, R} \rangle$: state space, actions, transition function, and rewards.
The file `mdp.jl` defines each of these using the `POMDPs.jl` "object-oriented" interface (this is typically how MDPs are defined using `POMDPs.jl`).
Please review this file.
We also include the file `viz.jl` for visualization purposes.

In [ ]:
#incude mdp.jl to define states, actions, transitions, and rewards. 
include("utils/mdp.jl");
include("utils/viz.jl");

Now let's take a few deterministic actions and visualize the grid world and the actions of the agent.

In [ ]:
step = Dict()
step[:start] = Point(1, 1)
step[:actions] = [N, E, E]
render(mdp, step)

Next, we will initialize our agent in the MDP, placing it at coordinates [1, 1].

In [ ]:
POMDPs.initialstate(mdp::GridWorld) = Deterministic(GridWorldState(1, 1))

Now, we will simulate the behavior of the agent with random policy: take random actions at each time step. You can see the agent meandering, including hitting against the boundaries. 

In [ ]:
mdp = GridWorld()

random_policy = RandomPolicy(mdp)

for (s, a, r) in stepthrough(mdp, random_policy, "s,a,r", max_steps=10)
    @show s
    @show a
    @show r
end

Now, we are ready to solve our MDP, for which we will use the `ValueIterationSolver` from the [DiscreteValueIteration](https://github.com/JuliaPOMDP/DiscreteValueIteration.jl) package. 

In [ ]:
# first let's load DiscreteValueIteration
using DiscreteValueIteration

# initialize the problem 
mdp = GridWorld()

# Try a slightly different initialization of the positive rewards and see how it impacts the behavior of the agent 
# mdp = GridWorld(rv=[-10.0, -5.0, 2, 10.0])

# initialize the solver
# max_iterations: maximum number of iterations value iteration runs for (default is 100)
# belres: the value of Bellman residual used in the solver (defualt is 1e-3)
solver = ValueIterationSolver(max_iterations=100, belres=1e-3; verbose=true)

# solve for an optimal policy
policy = solve(solver, mdp);

Now with our `policy` at hand, we can take actions using it. 

In [ ]:
POMDPs.initialstate(mdp::GridWorld) = Deterministic(GridWorldState(1, 1))
# record steps for visualization
step = Dict()
step[:start] = (1, 1)
step[:actions] = []
step[:visitedstates] = []
push!(step[:visitedstates], (1, 1))
for (s, a, r) in stepthrough(mdp, policy, "s,a,r", max_steps=10)
    push!(step[:actions], a)
    push!(step[:visitedstates], (s.pos.x, s.pos.y))
end
# visualize the world and the actions of the agent
render(mdp, step)

You can appreciate a much more goal-oriented behavior with the optimal policy.

Before we turn to the questions, there are two more critical functions we need to learn about in the `POMDPs.jl` package.

The function `action` consumes a `policy::Policy` and a current state (`state::GridWorldState` in our case). It returns an action (`action::GridWorldAction` in our case) under the given `policy`. (`action` is called, internally, by `stepthrough`.)

<div class="alert alert-warning" markdown="1">
    <strong> Note </strong>

The `action` function will only return the reward-maximizing `action` to take. 

</div>

Lastly, the function `value` consumes a `policy::Policy`, `state`, and `action` to compute the expected reward by taking the `action` in the given `state`.
_We will use `value` to create our Boltzman (or softmax) policy in our generative model._

In [ ]:
# say we are in state (9,2)
s = GridWorldState(9, 2)
a = action(policy, s)
println(a)
value(policy, s, E)

Write down the probability distribution that the transition function specifies in an MDP and describe in a sentence or two how `tprob` relates to the MDP's transition function.

YOUR ANSWER HERE

### 1B: [10 pts] Generative agent model

Write a generative model, called `agent_model`, that assumes the knowledge of the grid world defined above, except for the values of the positive rewards.

Place uniform (discrete) priors over these rewards, between 1 and 10.

Use the variable name `north_reward` and `south_reward` to refer to the two rewards with positive values on our grid world.

You must keep the values and locations of the negative rewards same as defined above. You must also keep the locations of the positive rewards same as above. 

Use the `ValueIterationSolver` to obtain a policy, and define your likelihood function by drawing actions following a Boltzman policy. Use a temperature of $\beta=1.5$. You can use the following function to turn the value vector into a softmax probability distribution.

In [ ]:
include("utils/draw.jl");

In [ ]:
function softmax(score; beta=1.5)
    score = score .- maximum(score)
    score = score ./ beta # temperature
    exp_score = exp.(score)
    return exp_score ./ sum(exp_score)
end

# we need to initialize the MDP in global context.
POMDPs.initialstate(mdp::GridWorld) = Deterministic(GridWorldState(1, 1))

<div class="alert alert-warning" markdown="1">
    <strong> Note </strong>


We use `Gen.@gen` (rather than `@gen`) because `POMDPs.jl` also exports an `@gen` macro.
Since we imported `Gen` via `using Gen`, we can prefix `Gen` to `@gen`, thus ensuring that our `agent_model` is parsed by `Gen.jl`, rather than `POMDPs.jl`.

</div>

In [ ]:
Gen.@gen function agent_model(maxsteps=10, beta=1.5)
    # draw random variables
    # your code here
    throw(Exception("Not Implemented."))

    # create the mdp with these random decisions
    # your code here
    throw(Exception("Not Implemented."))

    # use Value Iteration as your solver, and solve the MDP to get your policy
    # your code here
    throw(Exception("Not Implemented."))

    # We will use the dictionary step to keep track of the states and actions for visualization purposes
    step = Dict()
    step[:start] = (1, 1)
    step[:actions] = []
    step[:visitedstates] = []
    push!(step[:visitedstates], (1, 1))

    for (k, (s, a, r)) in enumerate(stepthrough(mdp, policy, "s,a,r", max_steps=maxsteps))
        # your code goes here, defining a Boltzman policy (keep beta=1.5). 
        # Your code should yield a probability vector over actions, called "prob_vector"
        # your code here
        throw(Exception("Not Implemented."))

        # Draw an action from the policy
        # your code here
        throw(Exception("Not Implemented."))

        # save the action and state
        # your code here
        throw(Exception("Not Implemented."))
    end

    return step, mdp
end
;

Draw a sample from your generative model, using the Gen.generate function. 

In [ ]:
# your code here
throw(Exception("Not Implemented."))
get_choices(trace)

Below, we provide a visualization code by utilizing the values we recorded and returned in the generative model.

In [ ]:
history_, mdp_ = get_retval(trace)
render(mdp_, history_)

### 1C: [10pts] Inference

In this section, we will pose an inference problem to our generative model; you will write an importance sampler and comment on the posterior it infers.

In [ ]:
# a helper function to make an observation choicemap.
function get_observations(trace; maxsteps=10)
    # Constrain the observed measurements.
    # your code here
    throw(Exception("Not Implemented."))
end;

You will pose an inference problem by synthesizing an action sequence from our generative model.
Sample a trace (`trace`) from your generative model where the `south_reward` is `3` and the `north_reward` is `7`.

In [ ]:
# your code here
throw(Exception("Not Implemented."))
;

Call the `get_observations` function to get a choicemap of the actions taken in the stimulated trace, called `observations`.

Then, visualize the trace.

<details class="alert alert-info" markdown="1">
    <summary><strong>Hint</strong></summary>

Similar visualizations have been provided in this problem set.
</details>

In [ ]:
# your code here
throw(Exception("Not Implemented."))

Now make inference using importance sampling by filling in the `do_inference` function. Return both the trace and its score. 

In [ ]:
function do_inference(amount_of_computation=500)
    # your code here
    throw(Exception("Not Implemented."))
    return (trace, score)
end;

Fill in the next code block to run this inference procedure for 5 chains, print out the positive reward values for each chain, and visualize the chain with the highest score.

In [ ]:
bestscore = -Inf
inferred_rewards = []
besttrace = nothing
for sample = 1:5
    # Call the do_inference functions
    # your code here
    throw(Exception("Not Implemented."))
    push!(inferred_rewards, (trace[:south_reward], trace[:north_reward]))
    if score > bestscore
        besttrace = trace
        bestscore = score
    end
end

println(inferred_rewards)

# visualize
# your code here
throw(Exception("Not Implemented."))

Comment on your posterior with 2 sentences: in what way does your posterior make sense? Your answer should consider the ill-posed nature of this inference problem.

YOUR ANSWER HERE